### I hope you've enjoyed working with datasets in pandas so far!

Now that you've become familiar with how to do some basic data wrangling on a single dataset in **Python ** using **pandas**, we are going to move on to more complicated data operations.

In this section we will cover the following:


1. Working with text:
  * Splitting a column into multiple pieces
  * Extracting text
  * Converting between data types
* Combining multiple datasets together coherently:
  * Joining/merging
* Gracefully working with (or removing) missing/bad/incomplete data
* Reshaping data:
  * Pivoting
  * Stacking/unstacking

Ok, let's get to it!

We are going to go back to the Movielens 1M dataset, but we are going to fill in a significant amount of detail that was excluded initially by including data that was stored in separate files.

Specifically, we will be filling in/filtering/transforming information about the movies and users found in the `ratings.dat` dataset we worked on by combining that original info with data found in the `movies.dat` and `users.dat` files. These can be found in the same folder as where `ratings.dat` is located.

So, lets begin!

In [1]:
%qtconsole
import pandas as pd
import numpy as np

As always, you have to import the modules you will/may need.

Here are the schemas for the two new files we will work with:

1. `movies.dat`:
  * Row format: MovieID::Title::Genres
    * `MovieID`: The id of the movie, should correspond to values in `MovieID` column found in `ratings.dat`
    * `Title`: The title, in plain english, of the movie
    * `Genres`: A list of genres the movie is associated with, can be of varying length per row
2. `users.dat`:
  * Row format: UserID::Gender::Age::Occupation::Zip-code
    * `UserID`: The id of the user, should correspond to values found in `UserID` column foind in `ratings.dat`
    * `Gender`: Self-reported gender of the user, either `M` or `F`
    * `Age`: Self-reported age of the user
    * `Occupation`: Self-reported occupation category of the user
    * `Zip-code`: Self-reported zip-code of the user

Lets load these files in, along with the first file, `ratings.dat`, that we worked with:

In [2]:
movieData  = pd.read_csv("./movieData/movies.dat",sep = "::", names = ["MovieID","Title","Genres"])
userData   = pd.read_csv("./movieData/users.dat",sep = "::", names = ["UserID","Gender","Age","Occupation","Zip-code"])
ratingData = pd.read_csv("./movieData/ratings.dat",sep = "::",names = ['UserID','MovieID','Rating','Timestamp'])

/usr/local/lib/python2.7/site-packages/pandas/io/parsers.py:639: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


Let's take a look at the first few rows in the `movies.dat` dataset, which we've stored in a variable called `movieData` and then lets make sure that every movie is uniquely represented in this set:

In [3]:
movieData.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
print movieData.shape
print len(movieData.MovieID.unique())

(3883, 3)
3883


So far, so good! There are as many rows in this dataset as there are unique movies!

**However**, we can clearly see that we will need to do some massaging with this dataset in order to get it into a more workable format: 
1. **Create a separate column for the year in which each movie was released by parsing the `Title` column (in case we want to do an analysis involving years)**
* **Parse the `Genres` column to extract every genre separately (this looks like it might be a bit tricky since there are different numbers of genres for each movie, but its actually really easy!)**

Lets try to tackle **1.** 

A reasonable approach here would be:

1. Creating a new column called `Year` by extracting the 4th-to-last through next-to-last characters from each Title 
* Converting this value from a `string` (or an `object`, as **pandas** calls them) into an `int`
* Removing the last 6 characters completely from the `Title` column

In [5]:
movieData["Year"] = movieData.Title.str.slice(-5,-1) #1
print movieData.head()
movieData.Year = movieData.Year.astype(int) #2
movieData.Title = movieData.Title.str.slice(0,-6) #3
print movieData.head()

   MovieID                               Title                        Genres  \
0        1                    Toy Story (1995)   Animation|Children's|Comedy   
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2        3             Grumpier Old Men (1995)                Comedy|Romance   
3        4            Waiting to Exhale (1995)                  Comedy|Drama   
4        5  Father of the Bride Part II (1995)                        Comedy   

   Year  
0  1995  
1  1995  
2  1995  
3  1995  
4  1995  
   MovieID                         Title                        Genres  Year
0        1                    Toy Story    Animation|Children's|Comedy  1995
1        2                      Jumanji   Adventure|Children's|Fantasy  1995
2        3             Grumpier Old Men                 Comedy|Romance  1995
3        4            Waiting to Exhale                   Comedy|Drama  1995
4        5  Father of the Bride Part II                         Comedy  19

Let's work through all 3 of these lines.

There's a whole slew of functions that you can use to operate on `string` values that **pandas** provides, and the way to access them is very similar to the way you accessed all the cool time functionality when you called `dt` on a given row in the previous lesson.

So, to access `str` functions for a given column, just call `DataFrameName.columnName.str`, which translates in our case to `movieData.Title.str`.

The function that we are using here is called `slice`:

`movieData["Year"] = movieData.Title.str.slice(-5,-1)`

`slice` allows us to extract a "slice" of the string out directly, based on the starting and ending index (position) of the values we want "sliced" out in relation to the whole string.

**Indices for lists, arrays, strings, and collections of values more generally in Python always begin at 0 and end at the length of the string/list/collection-1. Indexing can also proceed from the end of the string/list/collection, etc. backwards, using negative numbers.**

So, to extract the 4th-to-last to next-to-last values in the string, we have to use index values of -5 and -1.

There are other ways to extract or split/transform strings in **Python** and **pandas** and I encourage you to look over the other functions found in the string module of **Python** and the kinds of `string` operations built into **pandas** itself. [Here's a useful link](http://pandas.pydata.org/pandas-docs/stable/text.html)

The second line shouldn't seem nearly as challenging:

`movieData.Year = movieData.Year.astype(int)`

We are just taking the column, and forcing the values in the column to behave as `int` types (numbers) and not as `string` types (on which we can't operate numerically).

With that out of the way, lets tackle the next data massaging task:

**Parse the `Genres` column to extract every genre separately (this looks like it might be a bit tricky since there are different numbers of genres for each movie, but its actually not as hard as one would think!)**

The approach we are going to take here is fairly common in Data Science pipelines and involves converting categorical variables (like our genres column) into a number-like representation. 

This is typically called creating **indicator variables** (you'll see why they're called indicator variables shortly).

The way this approach works is:

1. **Collect all of the distinct values found for the given categorical type (in our case, genre).**
* **Create a column per category value, and indicate whether the given value is present or absent for each row in the dataset.**

The caveat here is that if your categorical variable has many distinct values that it ranges over (>100), then the transformation to indicator variables can lead to a really sparse, large, and space-inefficient representation of your data (for 100 distinct values, your matrix would have to be at least 99 columns wide).

Thankfully, **pandas** can do both steps **1. and 2.** for you in one line of code, as long as the data is stored in the right format (which it is!).

Here is how this transformation from categorical column -> indicator variable would be executed in **pandas**:

In [6]:
genresDF = movieData.Genres.str.get_dummies(sep = "|")
genresDF.head()

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


Awesome, so that totally converted the `Genres` column into the expanded indicator variable representation in one line of code!

Let's go through it:

`genresDF = movieData.Genres.str.get_dummies(sep = "|")`

We used `str.get_dummies()` on the `Genres` column and passed in a value to the `sep` parameter, telling the function that if a row had multiple entries, then the `|` character separated them. 

If, on the other hand, we only had a single genre per row, we could have left the parameter `sep` out completely and simply called:

`genresDF = movieData.Genres.str.get_dummies()`

But now we have a bit of a problem, all of these indicator variables are stored in a separate `DataFrame` from the original `movieData`. We can fix this by **joining the two datasets together.**

Join operations are very common operations in databases and in data processing pipelines more generally. **Join operations take two distinct datasets and combine them based on some common column or set of columns (called join keys) so that all columns from both datasets are combined into a single dataset based on the kind of join you want to perform.**

3 basic kinds of joins exist. To formalize the join types, lets say we have two datasets called A and B:

1. **One-sided (left or right) join**: This kind of join takes all the join keys from one of the datasets (lets say A), keeps all of them, and attempts to find all the rows in B with keys identical to only those in A. Any keys from A not found in B will have null values for columns in B.
* **Inner join**: This join only takes all the keys common to both A and B and combines all of the columns in both A and B together. Rows for which keys are not found in both A and B are thrown out.
* **Outer (full outer) join**: This join takes all the keys found in either A or B and combines both datasets on the keys that are common to both. For those keys that arent in both datasets, all columns remain, but have null values for the missing columns. (We will talk about null values soon).

I will not go into more detail about each of these kinds of joins now, but if youre interested in learning more about join types [this is a good explanation](http://blog.sweetlabs.com/2013/12/cheat-sheet-using-python-pandas-perform-fast-sql-like-joins/) and [here is the **pandas** documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html)

The exact type of join we will perform now actually doesn't matter, because all of them will lead to the same result (Do you know why?).

So, on to the join:

In [7]:
movieData = movieData.merge(genresDF,left_index=True,right_index=True)
movieData.head()

,MovieID,Title,Genres,Year,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,Animation|Children's|Comedy,1995,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,Adventure|Children's|Fantasy,1995,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,Comedy|Romance,1995,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,Comedy|Drama,1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,Comedy,1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


To perform a join, you can call **pandas** `merge()` function and pass it a variety of parameters (read the documentation for more info). In our case:

`movieData = movieData.merge(genresDF,left_index=True,right_index=True)`

We are calling `merge` from `movieData`, so `movieData` functions as our left dataset (table), and `genresDF` functions as our right table. 

We are using the indices of both tables as join keys (thats what setting the parameters `left_index` and `right_index` to `True` does, as by default they are `False`).

If you were to join on something other than the index of both tables (like another column), you would have passed that column name (or names) as a `string` or a `List` of `string` names to the `left_on` and `right_on` parameters.

One last thing to do with this dataset! 
We still have the `Genres` column in `movieData`, but we've expanded it and don't need it.
So, let's delete it from the `DataFrame`.

In [8]:
del movieData["Genres"]
movieData.head()

,MovieID,Title,Year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


Ok, we're done working with `movieData` for now, lets move on to `userData`:

In [9]:
userData.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


Here is some explanation about the `Age` and `Occupation` fields (before you think there is someone that is claiming to be a 1 year old in the dataset!):

- Age is a categorical variable mapped to the following ranges:

    *  1:  "Under 18"
    * 18:  "18-24"
    * 25:  "25-34"
    * 35:  "35-44"
    * 45:  "45-49"
    * 50:  "50-55"
    * 56:  "56+"

- Occupation is a categorical variable mapped as follows:

    * 0:  "other" or not specified
    * 1:  "academic/educator"
    * 2:  "artist"
    * 3:  "clerical/admin"
    * 4:  "college/grad student"
    * 5:  "customer service"
    * 6:  "doctor/health care"
    * 7:  "executive/managerial"
    * 8:  "farmer"
    * 9:  "homemaker"
    * 10: "K-12 student"
    * 11: "lawyer"
    * 12: "programmer"
    * 13: "retired"
    * 14: "sales/marketing"
    * 15: "scientist"
    * 16: "self-employed"
    * 17: "technician/engineer"
    * 18: "tradesman/craftsman"
    * 19: "unemployed"
    * 20: "writer"


Ok, so then there are several combinations of `Age` and `Occupation` that we can remove:

1. Make sure that every user mapped to `Age 1` has an `Occupation` `4 or 10`, if not, remove them.
*  Make sure no one that has an `Occupation` other than `10` is of `Age` either `1 or 18`, if they
* Make sure all of the zipcodes are in a standard (5 digit) format

Now that you're more familiar with **pandas**, you should figure out how to do all 3 of these and either remove or change the offending rows.
(**Hint:** Use `np.logical_and` to test for the occurence of multiple conditions for your mask that you are going to have to create! `np.logical_and` takes in 2 arguments, the two conditions you want satisfied).

Store the new, filtered `DataFrame` in a separate variable called `cleanedUserData` and keep the cleaned zipcodes in the column `Zip-code`.
(**Hint:** I suggest you use the slicing logic we used before for the `Title` column to extract the first 5 values in the `Zip-code` column and then convert the column from a `string` to an `int`)

In [19]:
#Some code
cleanedUserData = pass

0     48067
1     70072
2     55117
3     02460
4     55455
5     55117
6     06810
7     11413
8     61614
9     95370
10    04093
11    32793
12    93304
13    60126
14    22903
...
6025    11210
6026    20742
6027    94133
6028    23185
6029    32618
6030    45123
6031    55108
6032    78232
6033    94117
6034    78734
6035    32603
6036    76006
6037    14706
6038    01060
6039    11106
Name: Zip-code, Length: 6040, dtype: object

Ok, now that all of this data munging on both of these datasets is done, lets **join** them with the original `ratingsData`. 

To join each of these datasets, you will need to use a distinct set of **join keys:**

1. To join the `movieData` dataset to the `ratingData` dataset, you will need to join on the `MovieID` column.
2. To join the `cleanedUserData` dataset to the `ratingData` dataset, you will need to join on the `UserID` column.

Also, remember that we removed several bad rows from the `userData` to get the `cleanedUserData` so when we perform the default **join** (an *inner join*) with the `ratingData` we will get a smaller dataset out!

So, go ahead and perform the joins yourself and store the resulting, final joined dataset in a variable called `filledAndCleanedRatingData`

In [13]:
#Some code
#filledAndCleanedRatingData = ratingData.merge(userData,on = "UserID")
#filledAndCleanedRatingData.head()
#filledAndCleanedRatingData = filledAndCleanedRatingData.merge(movieData,on = "MovieID")
filledAndCleanedRatingData.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Year,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest,1975,...,0,0,0,0,0,0,0,0,0,0
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest,1975,...,0,0,0,0,0,0,0,0,0,0
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest,1975,...,0,0,0,0,0,0,0,0,0,0
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest,1975,...,0,0,0,0,0,0,0,0,0,0
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest,1975,...,0,0,0,0,0,0,0,0,0,0


And now, just to get a bit more practice with munging and transforming data in pandas, try to answer the following questions:

1. What was the most frequently rated movie?
  * In january?
* What genre was most popular (by avg. rating) in winter (January-March)?
  * What genre was the most frequently rated?
* What movie was most commonly rated in the mornings (before noon)?
* Which user saw movies that belonged to the largest collection of distinct genres?
* Which genre had the most varied ratings?
* Which movie did the most people disagree on (had the highest standard deviation in ratings)?

In [ ]:
#Write your code here

You should now be very comfortable with some of the more common